## Code repository for building global climate change mitigation measures dataset

This notebook comprises the methodology and considerations for sourcing database entries in the global climate change mitigation measures dataset.  The dataset is an aggregation of publicly documented climate change mitigation measures from around the world. The measures are sourced from multiple databases spanning efforts on the part of government and non-governmental agencies across a range of fields.   

More information on each source and its original formatting can be found <a href="https://github.com/Dlindse/gccmdb/blob/master/GCCMDB%20Source%20List.pdf">here</a>

Dataset sources include:

International Energy Agency:
- Addressing Climate Change Database
- IEA/IRENA Global Renewable Energy Policies and Measures Database
- Energy Efficiency Database

European Environment Agency
-  Climate Change Mitigation Policies and Measures

Mesures d'Utilisation Rationnelle de l'Energie (MURE)

Private Participation in Renewable Energy (PPI-RE)

Climate Change Laws of the World

- Legislation database

Database of State Incentives for Renewables & Efficiency (DSIRE)

World Energy Council Energy Efficiency Policy and Measures

#### A note on dataset sourcing methodology and invitation to contribute:

Wherever stated in the source databases we have provided the correct attributions. In building the dataset of global climate change mitigation measures we aim to be as inclusive, comprehensive, and accurate as possible in our compilation of documentation. However, there is no claim to have identified every documented measure, policy instrument or project in the countries included in the database.

We invite any amendments, updates, or new submissions to the collection here. 

### Setting the environment

In [1]:
#import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get
import json
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import datetime

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import asyncio
import concurrent.futures
import re
import os
import nltk

#from textblob import Word
from nltk.tag import pos_tag
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk import download as nltk_download
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.datasets.twenty_newsgroups import fetch_20newsgroups
""

nltk.download('stopwords')
stemmer = SnowballStemmer("english")
stop_words=stopwords.words('english')

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)




[nltk_data] Downloading package stopwords to /Users/d/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## International Energy Agency Databases



### How this dataset was originated:

Since 1999, the IEA’s Policies and Measures Databases offer access to information on energy-related policies and measures taken or planned to reduce greenhouse gas emissions, improve energy efficiency and support renewable energy development and deployment. Delegates from IEA member countries are given the opportunity to review information in the databases twice a year. The IEA consults with country representatives prior to including new information on the public version of this site.<sup>1</sup>

___

<sup>1</sup>“IEA” IEA Policies and Measures Database. Retrieved September 5, 2019 (https://www.iea.org/policiesandmeasures/)


### How we got this data:

To source this data we used web requests and BeautifulSoup at each individual database to parse out the country list and then construct links to each country's index of measures. We compiled a complete list of entries by country for each database and then used multi threaded concurrent web requests to parse each entry. 

In [29]:
#get all countries with entiries in the IEA database
get_countries = "https://www.iea.org/policiesandmeasures/climatechange/"
g_c = get(get_countries)

soup_c = BeautifulSoup(g_c.text, 'html.parser')
c_list = soup_c.find("ul", class_ ="listexpander")
c_loop = c_list.find_all("label")

country_store = []
for c in c_loop:
    country_store.append(c.text)
    
full_list = country_store[4:]
print(full_list)

['Albania', 'Algeria', 'Australia', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Brazil', 'Canada', 'China', 'Croatia', 'Czech Republic', 'Denmark', 'Estonia', 'European Union', 'Finland', 'France', 'Germany', 'Ghana', 'Greece', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Ireland', 'Israel', 'Italy', 'Japan', 'Korea', 'Latvia', 'Luxembourg', 'Malaysia', 'Mauritius', 'Mexico', 'Mongolia', 'Montenegro', 'Netherlands', 'New Zealand', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'Slovak Republic', 'South Africa', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom', 'United States']


In [30]:
len(country_store[4:])

51

In [34]:
#get each countries directory of database entries for climate policies and measure, renewable energy, 
#and energy efficiency
def get_countries(link_store, url):
    for country in full_list:
        if " " in country:
            amend = country.replace(" ", "%20")
            policy_link = url + amend 
            link_store.append(policy_link)
        else:
            policy_link = url + country
            link_store.append(policy_link)

In [7]:
#get individual links for each entry in db

def store_pages(store, ptype):
    for p in ptype:
        g_p = session.get(p)
        soup_p = BeautifulSoup(g_p.text, 'html.parser')
        entries = soup_p.find_all("tr")[1:]
        for entry in entries:
            ext = entry.find("a", href = True)
            link = "https://www.iea.org/" + ext._attr_value_as_string("href")
            store.append(link)



In [31]:
#for climate policies and measure database
policy_link_store = []
get_countries(policy_link_store, "https://www.iea.org/policiesandmeasures/climatechange/?country=")
print(len(policy_link_store))

51


In [35]:
#get individual links for each policy and measure page for each country          
page_link_store = []
store_pages(page_link_store, policy_link_store) 
print(len(page_link_store))

2790


In [32]:
#for renewable energy
renewables_store = []
get_countries(renewables_store, "https://www.iea.org/policiesandmeasures/renewableenergy/?country=")
print(len(renewables_store))

51


In [36]:
#get individual links for each renewable energy policy for each country 
renewpage_link_store = []
store_pages(renewpage_link_store,renewables_store)
print(len(renewpage_link_store))

1494


In [33]:
#energy efficiency
efficiency_store = []
get_countries(efficiency_store, "https://www.iea.org/policiesandmeasures/energyefficiency/?country=")
print(len(efficiency_store))


51


In [37]:
#get individual links for each energy efficiency policy for each country 
eff_page_link_store = []
store_pages(eff_page_link_store, efficiency_store)
print(len(eff_page_link_store))

1995


In [28]:
#test sections
#test link

entry ={}

testLink = "https://www.iea.org/policiesandmeasures/pams/austria/name-157022-en.php?s=dHlwZT1jYyZzdGF0dXM9T2s,&return=PG5hdiBpZD0iYnJlYWRjcnVtYiI-PGEgaHJlZj0iLyI-SG9tZTwvYT4gJnJhcXVvOyA8YSBocmVmPSIvcG9saWNpZXNhbmRtZWFzdXJlcy8iPlBvbGljaWVzIGFuZCBNZWFzdXJlczwvYT4gJnJhcXVvOyA8YSBocmVmPSIvcG9saWNpZXNhbmRtZWFzdXJlcy9jbGltYXRlY2hhbmdlLyI-Q2xpbWF0ZSBDaGFuZ2U8L2E-PC9uYXY-"
response = session.get(testLink)
soup_e = BeautifulSoup(response.text, 'html.parser')
title = soup_e.find(id="banner").get_text()
table = soup_e.find("tbody").find_all("tr")
column = [t.find_all("td") for t in table]

#print(title)
entry["Name:"] = title

for t in column:
    
    header = t[0].get_text()
    
    text = t[1].get_text()
    if text == "":
        text = np.nan
    if header not in ["Country:", "Year:", "Policy status:", "Jurisdiction:", "Date Effective:", "Policy Type:", "URL:", "Description:", "Policy Target:", "Technology Target:"]:
        print("Undocumented!", header)
        
    if (header == "Country:" or 
        header == "Year:" or
        header == "Policy status:" or
        header == "Jurisdiction:" or
        header == "Date Effective:" or
        header == "Policy Type:" or
        header == "Policy Target:" or
        header == "URL:" or
        header == "Description:" or
        header == "Technology Target:"):
        
            if entry.get('URL:', 0) !=0 and header == "URL:":
                pass
            
            else:
            
                entry[header] = text
        

        
print(entry)
        
        
    
    

#print(column[0:])

{'Name:': 'Green book for an integrated energy- and climate strategy', 'Country:': 'Austria', 'Year:': '2016', 'Policy status:': 'In Force', 'Jurisdiction:': 'National', 'Date Effective:': '2016', 'Policy Type:': 'Policy Support>Strategic planning', 'Policy Target:': 'Framework/ Multi-sectoral Policy', 'URL:': 'http://www.konsultation-energie-klima.at/assets/Uploads/Grunbuch-integrierte-Energiestrategie.pdf\t\t\t', 'Description:': 'The publication of the Green book in June 2016 has started the discussion for an integrated energy- and climate strategy. The green book analyses the current situation and compares existing scenarios. A set of questions kick starts the public debate, which includes target setting for 2030 and visioning till 2050.'}


###### Addressing Climate Change Database

In [58]:
#Scrape each climate policy webpage in db
climate_set = []
optional = list()

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in page_link_store
        ]
        
        for response in await asyncio.gather(*futures):
            entry = dict()            
            soup_e = BeautifulSoup(response.text, 'html.parser')
            title = soup_e.find(id="banner").get_text()
            table = soup_e.find("tbody").find_all("tr")
            column = [t.find_all("td") for t in table]
            #print(title)
            entry["Name:"] = title
            for t in column:
                if len(t) == 2:
                    header = t[0].get_text()    
                    text = t[1].get_text()
                    if text is "":
                        text = np.nan
                        
#                        print("Undocumented!", header) 
                        optional.append(header)
                        
                    if (header == "Country:" or 
                        header == "Year:" or
                        header == "Policy status:" or
                        header == "Jurisdiction:" or
                        header == "Date Effective:" or
                        header == "Policy Type:" or
                        header == "Policy Target:" or
                        header == "URL:" or
                        header == "Description:" or
                        header == "Technology Target:" or
                        header == 'Agency:'or
                        header == 'Amended:' or
                        header == 'Date Amended:' or
                        header == 'Date Ended:' or
                        header == 'Enforcement:' or
                        header == 'Evaluation:' or
                        header == 'Funding:' or
                        header == 'Legal References:' or
                        header == 'Penalty:' or
                        header == 'Related Documents:' or
                        header == 'Related policies:' or
                        header == 'This record is superseded by:' or
                        header == 'This record supersedes:' or
                        header == 'Funding:'):

                            if entry.get('URL:', 0) != 0 and header == "URL:":
                                pass
                            else:
                                entry[header] = text
                                                
                elif len(t) == 1:
                    text = np.nan
                        
#                    if header not in ["Country:", "Year:", "Policy status:", "Jurisdiction:", "Date Effective:", "Policy Type:", "URL:", "Description:", "Policy Target:", "Technology Target:"]:
#                        print("Undocumented!", header)
#                        optional.append(header)
                        
                    if (header == "Country:" or 
                        header == "Year:" or
                        header == "Policy status:" or
                        header == "Jurisdiction:" or
                        header == "Date Effective:" or
                        header == "Policy Type:" or
                        header == "Policy Target:" or
                        header == "URL:" or
                        header == "Description:" or
                        header == "Technology Target:" or
                        header == 'Agency:'or
                        header == 'Amended:' or
                        header == 'Date Amended:' or
                        header == 'Date Ended:' or
                        header == 'Enforcement:' or
                        header == 'Evaluation:' or
                        header == 'Funding:' or
                        header == 'Legal References:' or
                        header == 'Penalty:' or
                        header == 'Related Documents:' or
                        header == 'Related policies:' or
                        header == 'This record is superseded by:' or
                        header == 'This record supersedes:' or
                        header == 'Funding:'):

                            if entry.get('URL:', 0) !=0 and header == "URL:":
                                pass

                            else:

                                entry[header] = text    
                        
                        
        
            entry["data_source"] = "IEA"
            
            columns_complete = set(["Country:", 
                                   "Year:", 
                                   "Policy status:", 
                                   "Jurisdiction:", 
                                   "Date Effective:", 
                                   "Policy Type:", 
                                   "URL:", 
                                   "Description:", 
                                   "Policy Target:", 
                                   "Technology Target:",
                                   'Agency:',
                                   'Amended:',
                                   'Date Amended:',
                                   'Date Ended:',
                                   'Enforcement:',
                                   'Evaluation:',
                                   'Funding:',
                                   'Legal References:',
                                   'Penalty:',
                                   'Related Documents:',
                                   'Related policies:',
                                   'This record is superseded by:',
                                   'This record supersedes:',
                                   'Funding:',
                                   'Legal References:'])
            
            missing = columns_complete - set(entry.keys())
            
            for c in missing:
                entry[c] = np.nan

            
            
            climate_set.append(entry)



loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [54]:
set(optional)

{'Amended:', 'Description:', 'Policy Target:', 'Policy Type:', 'Year:'}

In [56]:
len(climate_set)

2790

In [57]:
climate_set[3]

{'Agency:': 'Tasmania Department of Premier and Cabinet\t\t\t\r\n',
 'Amended:': nan,
 'Country:': 'Australia',
 'Date Amended:': nan,
 'Date Effective:': '2017',
 'Date Ended:': nan,
 'Description:': "Climate Action 21 sets the Tasmanian Government’s agenda for action on climate change through to 2021. It reflects the Tasmanian Government's commitment to addressing the critical issue of climate change and articulates how Tasmania will play its role in the global response to climate change.\nClimate Action 21 is structured into six priorities. Each priority area includes a 2021 vision statement, our actions for 2017 to 2021 and details of initiatives already being progressed by the Tasmanian Government. Climate Action 21 has 37 actions.\nThe Climate Action 21 sets Tasmania on a pathway to reach a target of zero net emissions by 2050. The 2050 Tasmania’s emissions target means that the region’s emissions will be balanced out by our carbon sinks.",
 'Enforcement:': nan,
 'Evaluation:': n

######   IEA/IRENA Global Renewable Energy Policies and Measures Database

In [59]:
#Scrape each renewable energy policy webpage in db


renew_set = []
optional = list()

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in renewpage_link_store
        ]
        
        for response in await asyncio.gather(*futures):
            entry = dict()            
            soup_e = BeautifulSoup(response.text, 'html.parser')
            title = soup_e.find(id="banner").get_text()
            table = soup_e.find("tbody").find_all("tr")
            column = [t.find_all("td") for t in table]
            #print(title)
            entry["Name:"] = title
            for t in column:
                if len(t) == 2:
                    header = t[0].get_text()    
                    text = t[1].get_text()
                    if text is "":
                        text = np.nan
                        
#                        print("Undocumented!", header) 
                        optional.append(header)
                        
                    if (header == "Country:" or 
                        header == "Year:" or
                        header == "Policy status:" or
                        header == "Jurisdiction:" or
                        header == "Date Effective:" or
                        header == "Policy Type:" or
                        header == "Policy Target:" or
                        header == "URL:" or
                        header == "Description:" or
                        header == "Technology Target:" or
                        header == 'Agency:'or
                        header == 'Amended:' or
                        header == 'Date Amended:' or
                        header == 'Date Ended:' or
                        header == 'Enforcement:' or
                        header == 'Evaluation:' or
                        header == 'Funding:' or
                        header == 'Legal References:' or
                        header == 'Penalty:' or
                        header == 'Related Documents:' or
                        header == 'Related policies:' or
                        header == 'This record is superseded by:' or
                        header == 'This record supersedes:' or
                        header == 'Funding:'):

                            if entry.get('URL:', 0) != 0 and header == "URL:":
                                pass
                            else:
                                entry[header] = text
                                                
                elif len(t) == 1:
                    text = np.nan
                        
#                    if header not in ["Country:", "Year:", "Policy status:", "Jurisdiction:", "Date Effective:", "Policy Type:", "URL:", "Description:", "Policy Target:", "Technology Target:"]:
#                        print("Undocumented!", header)
#                        optional.append(header)
                        
                    if (header == "Country:" or 
                        header == "Year:" or
                        header == "Policy status:" or
                        header == "Jurisdiction:" or
                        header == "Date Effective:" or
                        header == "Policy Type:" or
                        header == "Policy Target:" or
                        header == "URL:" or
                        header == "Description:" or
                        header == "Technology Target:" or
                        header == 'Agency:'or
                        header == 'Amended:' or
                        header == 'Date Amended:' or
                        header == 'Date Ended:' or
                        header == 'Enforcement:' or
                        header == 'Evaluation:' or
                        header == 'Funding:' or
                        header == 'Legal References:' or
                        header == 'Penalty:' or
                        header == 'Related Documents:' or
                        header == 'Related policies:' or
                        header == 'This record is superseded by:' or
                        header == 'This record supersedes:' or
                        header == 'Funding:'):

                            if entry.get('URL:', 0) !=0 and header == "URL:":
                                pass

                            else:

                                entry[header] = text    
                        
                        
        
            entry["data_source"] = "IEA"
            
            columns_complete = set(["Country:", 
                                   "Year:", 
                                   "Policy status:", 
                                   "Jurisdiction:", 
                                   "Date Effective:", 
                                   "Policy Type:", 
                                   "URL:", 
                                   "Description:", 
                                   "Policy Target:", 
                                   "Technology Target:",
                                   'Agency:',
                                   'Amended:',
                                   'Date Amended:',
                                   'Date Ended:',
                                   'Enforcement:',
                                   'Evaluation:',
                                   'Funding:',
                                   'Legal References:',
                                   'Penalty:',
                                   'Related Documents:',
                                   'Related policies:',
                                   'This record is superseded by:',
                                   'This record supersedes:',
                                   'Funding:',
                                   'Legal References:'])
            
            missing = columns_complete - set(entry.keys())
            
            for c in missing:
                entry[c] = np.nan

            
            
            renew_set.append(entry)



loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [60]:
set(optional)

{'Description:', 'Policy Sector:', 'Policy Target:', 'Policy Type:', 'Year:'}

In [61]:
len(renew_set)

1494

In [62]:
renew_set[1]

{'Agency:': 'Ministry of Industry, Energy and Tourism (Minetur)',
 'Amended:': nan,
 'Country:': 'Albania',
 'Date Amended:': nan,
 'Date Effective:': '2014 (Nov 1st)',
 'Date Ended:': nan,
 'Description:': 'Order IET/1882/2014 of 14 October establishes the methodology for calculating the electricity generated by means of non-renewable fuels in STE and hybrid installations (classified in article 2 of RD 413/2014 as b.1.2. and in article 4 of RD 413/2014 as type 2 hybrid installations)/',
 'Enforcement:': nan,
 'Evaluation:': nan,
 'Funding:': nan,
 'Jurisdiction:': 'National',
 'Legal References:': 'Order IET/1882/2014 of 14 October; Royal Decree 413/2014',
 'Name:': 'Order IET/1882/2014 of 14 October. Electricity generated by means of non-renewable fuels in STE  and hybrid installations',
 'Penalty:': nan,
 'Policy Target:': 'Solar Thermal>Solar thermal electricity (STE)',
 'Policy Type:': 'Economic Instruments>Market-based instruments',
 'Policy status:': 'In Force',
 'Related Docume

###### Energy Efficiency Database

In [64]:
#Scrape each energy efficiency policy webpage in db
eff_set = []
optional = list()



async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in eff_page_link_store
        ]
        
        for response in await asyncio.gather(*futures):
            entry = dict()            
            soup_e = BeautifulSoup(response.text, 'html.parser')
            title = soup_e.find(id="banner").get_text()
            table = soup_e.find("tbody").find_all("tr")
            column = [t.find_all("td") for t in table]
            #print(title)
            entry["Name:"] = title
            for t in column:
                if len(t) == 2:
                    header = t[0].get_text()    
                    text = t[1].get_text()
                    if text is "":
                        text = np.nan
                        
#                        print("Undocumented!", header) 
                        optional.append(header)
                        
                    if (header == "Country:" or 
                        header == "Year:" or
                        header == "Policy status:" or
                        header == "Jurisdiction:" or
                        header == "Date Effective:" or
                        header == "Policy Type:" or
                        header == "Policy Target:" or
                        header == "URL:" or
                        header == "Description:" or
                        header == "Technology Target:" or
                        header == 'Agency:'or
                        header == 'Amended:' or
                        header == 'Date Amended:' or
                        header == 'Date Ended:' or
                        header == 'Enforcement:' or
                        header == 'Evaluation:' or
                        header == 'Funding:' or
                        header == 'Legal References:' or
                        header == 'Penalty:' or
                        header == 'Related Documents:' or
                        header == 'Related policies:' or
                        header == 'This record is superseded by:' or
                        header == 'This record supersedes:' or
                        header == 'Funding:'):

                            if entry.get('URL:', 0) != 0 and header == "URL:":
                                pass
                            else:
                                entry[header] = text
                                                
                elif len(t) == 1:
                    text = np.nan
                        
#                    if header not in ["Country:", "Year:", "Policy status:", "Jurisdiction:", "Date Effective:", "Policy Type:", "URL:", "Description:", "Policy Target:", "Technology Target:"]:
#                        print("Undocumented!", header)
#                        optional.append(header)
                        
                    if (header == "Country:" or 
                        header == "Year:" or
                        header == "Policy status:" or
                        header == "Jurisdiction:" or
                        header == "Date Effective:" or
                        header == "Policy Type:" or
                        header == "Policy Target:" or
                        header == "URL:" or
                        header == "Description:" or
                        header == "Technology Target:" or
                        header == 'Agency:'or
                        header == 'Amended:' or
                        header == 'Date Amended:' or
                        header == 'Date Ended:' or
                        header == 'Enforcement:' or
                        header == 'Evaluation:' or
                        header == 'Funding:' or
                        header == 'Legal References:' or
                        header == 'Penalty:' or
                        header == 'Related Documents:' or
                        header == 'Related policies:' or
                        header == 'This record is superseded by:' or
                        header == 'This record supersedes:' or
                        header == 'Funding:'):

                            if entry.get('URL:', 0) !=0 and header == "URL:":
                                pass

                            else:

                                entry[header] = text    
                        
                        
        
            entry["data_source"] = "IEA"
            
            columns_complete = set(["Country:", 
                                   "Year:", 
                                   "Policy status:", 
                                   "Jurisdiction:", 
                                   "Date Effective:", 
                                   "Policy Type:", 
                                   "URL:", 
                                   "Description:", 
                                   "Policy Target:", 
                                   "Technology Target:",
                                   'Agency:',
                                   'Amended:',
                                   'Date Amended:',
                                   'Date Ended:',
                                   'Enforcement:',
                                   'Evaluation:',
                                   'Funding:',
                                   'Legal References:',
                                   'Penalty:',
                                   'Related Documents:',
                                   'Related policies:',
                                   'This record is superseded by:',
                                   'This record supersedes:',
                                   'Funding:',
                                   'Legal References:'])
            
            missing = columns_complete - set(entry.keys())
            
            for c in missing:
                entry[c] = np.nan

            
            
            eff_set.append(entry)



loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [65]:
len(eff_set)

1995

In [66]:
set(optional)

{'Description:', 'Policy Target:', 'Policy Type:', 'Year:'}

In [67]:
eff_set[8]

{'Agency:': 'Department of Industry',
 'Amended:': nan,
 'Country:': 'Australia',
 'Date Amended:': nan,
 'Date Effective:': '2012',
 'Date Ended:': '2014',
 'Description:': 'The Local Government Energy Efficiency Program (LGEEP) was a non-competitive grant program that will assists local governing authorities (LGAs) install solar and heat pump hot water systems to drive smarter energy use in their buildings and community facilities.\xa0',
 'Enforcement:': nan,
 'Evaluation:': nan,
 'Funding:': 'AUD 6.8 million',
 'Jurisdiction:': 'National',
 'Legal References:': nan,
 'Name:': 'Local Government Energy Efficiency Program (LGEEP)',
 'Penalty:': nan,
 'Policy Target:': 'Buildings>Building Type>Non-residential, Residential Appliances>Water heating',
 'Policy Type:': 'Economic Instruments>Fiscal/financial incentives>Grants and subsidies, Economic Instruments>Direct investment>Funds to sub-national governments',
 'Policy status:': 'Ended',
 'Related Documents:': nan,
 'Related policies:': 

In [68]:
print('climate change policy db:', len(climate_set))
print('renewable energy policy db:', len(renew_set))
print('energy efficiency db:', len(eff_set))

full_set = climate_set + renew_set + eff_set

print('total:', len(full_set))

climate change policy db: 2790
renewable energy policy db: 1494
energy efficiency db: 1995
total: 6279


In [128]:
iea_db = pd.DataFrame(full_set)

In [129]:
iea_db

,Agency:,Amended:,Country:,Date Amended:,Date Effective:,Date Ended:,Description:,Enforcement:,Evaluation:,Funding:,...,Policy Type:,Policy status:,Related Documents:,Related policies:,Technology Target:,This record is superseded by:,This record supersedes:,URL:,Year:,data_source
0,NaN,NaN,Albania,NaN,2015,NaN,The 21st session of the Conference of the Part...,NaN,NaN,NaN,...,Policy Support>Strategic planning,In Force,NaN,NaN,NaN,NaN,NaN,http://www4.unfccc.int/ndcregistry/PublishedDo...,2015,IEA
1,"Ministry of Industry and Energy, National Agen...",NaN,Albania,NaN,2003,NaN,The main goal of the Albanian National Strateg...,NaN,NaN,NaN,...,Policy Support>Strategic planning,In Force,NaN,NaN,NaN,NaN,NaN,http://unfccc.int/files/meetings/seminar/appli...,2003,IEA
2,Department of Agriculture,NaN,Australia,NaN,8 December 2011,2014,The Carbon Farming Initiative (CFI) was a volu...,NaN,NaN,NaN,...,Economic Instruments>Market-based instruments>...,Superseded,NaN,"Carbon Farming Futures\r\n, Carbon Farming Ski...",NaN,Emissions Reduction Fund\n,NaN,http://www.agriculture.gov.au/climatechange/ca...,8 December 2011,IEA
3,Tasmania Department of Premier and Cabinet\t\t...,NaN,Australia,NaN,2017,NaN,Climate Action 21 sets the Tasmanian Governmen...,NaN,NaN,AUD 3m (in addition to $400m already invested ...,...,"Policy Support>Strategic planning, Information...",In Force,NaN,NaN,NaN,NaN,NaN,http://www.dpac.tas.gov.au/divisions/climatech...,2017,IEA
4,Department of the Environment and Energy,NaN,Australia,NaN,2016,NaN,The $5 million Solar Communities program will ...,NaN,NaN,AUD$5 million,...,Economic Instruments>Fiscal/financial incentiv...,In Force,NaN,NaN,NaN,NaN,NaN,http://www.environment.gov.au/climate-change/r...,2016,IEA
5,Queensland Government,NaN,Australia,NaN,2016,NaN,"The Queensland Government, in partnership with...",NaN,NaN,AUD$12 million,...,Economic Instruments>Fiscal/financial incentiv...,In Force,NaN,NaN,NaN,NaN,NaN,https://www.qld.gov.au/environment/climate/ada...,2016,IEA
6,Department of the Environment and Energy,NaN,Australia,NaN,2015-2016,NaN,Develop a formal government strategy and imple...,NaN,NaN,NaN,...,"Regulatory Instruments>Codes and standards, Re...",Ended,NaN,Australian Renewable Energy Agency (ARENA)\n,NaN,NaN,NaN,http://www.environment.gov.au,2015-2016,IEA
7,Department of the Environment and Energy\r\n\r\n,NaN,Australia,NaN,2015,NaN,The Clean Energy Finance Corporation (CEFC) wa...,NaN,NaN,$2 billion AUD per annum for 5 years commencin...,...,Economic Instruments>Fiscal/financial incentiv...,In Force,NaN,Australian Renewable Energy Agency (ARENA)\n,NaN,NaN,NaN,http://www.cleanenergyfinancecorp.com.au,2015,IEA
8,Department of the Environment and Energy,NaN,Australia,NaN,2015,NaN,In line with the Government's commitment to re...,NaN,NaN,NaN,...,Regulatory Instruments>Codes and standards,In Force,NaN,NaN,NaN,NaN,NaN,http://www.environment.gov.au,2015,IEA
9,Department of the Environment and Energy,NaN,Australia,NaN,2015,NaN,Australia’s post-2020 emission reduction targe...,NaN,NaN,NaN,...,Policy Support>Strategic planning,In Force,NaN,Nationally Determined Contribution (NDC) to th...,NaN,NaN,NaN,https://www.environment.gov.au/climate-change/...,2015,IEA


In [130]:
list(iea_db.columns)

['Agency:',
 'Amended:',
 'Country:',
 'Date Amended:',
 'Date Effective:',
 'Date Ended:',
 'Description:',
 'Enforcement:',
 'Evaluation:',
 'Funding:',
 'Jurisdiction:',
 'Legal References:',
 'Name:',
 'Penalty:',
 'Policy Target:',
 'Policy Type:',
 'Policy status:',
 'Related Documents:',
 'Related policies:',
 'Technology Target:',
 'This record is superseded by:',
 'This record supersedes:',
 'URL:',
 'Year:',
 'data_source']

In [131]:
iea_db.rename(columns={'Name:': 'name', 
                       'Country:':'country',
                       'Agency:' : 'agency',
                         'Date Effective:': 'policy_implementation_period_start_year',
                         'Date Ended:': 'policy_implementation_period_end_year',
                         'Description:': 'description',
                         'Evaluation:' : 'impact_evaluation',
                         'Funding:' : 'policy_financing_quantity',
                         'Jurisdiction:': 'jurisdiction',
                         'Policy Target:': 'policy_target',
                         'Policy Type:' : 'policy_type',
                         'Policy status:' : 'policy_status',
                         'Related Documents:' : 'related_documents',
                         'URL:' : 'link',
                         'Year:' : 'policy_written_year' }, inplace=True)

list(iea_db.columns)


['agency',
 'Amended:',
 'country',
 'Date Amended:',
 'policy_implementation_period_start_year',
 'policy_implementation_period_end_year',
 'description',
 'Enforcement:',
 'impact_evaluation',
 'policy_financing_quantity',
 'jurisdiction',
 'Legal References:',
 'name',
 'Penalty:',
 'policy_target',
 'policy_type',
 'policy_status',
 'related_documents',
 'Related policies:',
 'Technology Target:',
 'This record is superseded by:',
 'This record supersedes:',
 'link',
 'policy_written_year',
 'data_source']

In [132]:
#combine 'Technology Target:' + 'Policy Target:'

iea_db['policy_target'] = iea_db['policy_target'] + iea_db['Technology Target:']

In [133]:
#drop other columns

iea_db = iea_db.drop(['Legal References:', 'Amended:', 'Date Amended:', 'Enforcement:', 'Penalty:', 'Related policies:', 'This record is superseded by:', 'This record supersedes:', 'Technology Target:'], axis=1)

list(iea_db.columns)


['agency',
 'country',
 'policy_implementation_period_start_year',
 'policy_implementation_period_end_year',
 'description',
 'impact_evaluation',
 'policy_financing_quantity',
 'jurisdiction',
 'name',
 'policy_target',
 'policy_type',
 'policy_status',
 'related_documents',
 'link',
 'policy_written_year',
 'data_source']

In [145]:
dropped_columns = ['Legal References:', 'Amended:', 'Date Amended:', 'Enforcement:', 'Penalty:', 'Related policies:', 'This record is superseded by:', 'This record supersedes:', 'Technology Target:']


print("Dropped columns:", ", ".join(dropped_columns))



Dropped columns: Legal References:, Amended:, Date Amended:, Enforcement:, Penalty:, Related policies:, This record is superseded by:, This record supersedes:, Technology Target:


In [146]:
# add the rest

iea_db['world_region'] = np.nan
iea_db['country_region'] = np.nan
iea_db['city'] = np.nan
iea_db['ghgs_affected'] = np.nan
iea_db['related_to_eu_policy_boolean'] = np.nan
iea_db['related_to_eu_policy'] = np.nan
iea_db['impact_on_eu_ets_esd_or_lulucf_emissions'] = np.nan
iea_db['total_ghg_emissions_reductions_in_2020'] = np.nan
iea_db['total_ghg_emissions_reductions_in_2030'] = np.nan
iea_db['related_to_neeap_policy_boolean'] = np.nan
iea_db['related_to_neeap_policy'] = np.nan
iea_db['policy_financing_period_start'] = np.nan
iea_db['policy_financing_period_end'] = np.nan
iea_db['percent_private_funded'] = np.nan

list(iea_db.columns)
len(list(iea_db.columns))

30

In [135]:
#to csv
iea_db.to_csv(path_or_buf='data/iea_db.csv', index=False)

In [122]:
iea = pd.read_csv('data/iea_db.csv')
iea.columns

Index(['agency', 'country', 'policy_implementation_period_start_year',
       'policy_implementation_period_end_year', 'description',
       'impact_evaluation', 'policy_financing_quantity', 'jurisdiction',
       'name', 'policy_target', 'policy_type', 'policy_status',
       'related_documents', 'link', 'policy_written_year', 'data_source',
       'world_region', 'country_region', 'city', 'ghgs_affected',
       'related_to_eu_policy_boolean', 'related_to_eu_policy',
       'impact_on_eu_ets_esd_or_lulucf_emissions',
       'total_ghg_emissions_reductions_in_2020',
       'total_ghg_emissions_reductions_in_2030',
       'related_to_neeap_policy_boolean', 'related_to_neeap_policy',
       'policy_financing_period_start', 'policy_financing_period_end',
       'percent_private_funded'],
      dtype='object')

In [242]:
iea_countries = iea['country'].tolist()
set(iea_countries)

{'Albania',
 'Algeria',
 'Australia',
 'Austria',
 'Belgium',
 'Bosnia and Herzegovina',
 'Brazil',
 'Canada',
 'China',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'European Union',
 'Finland',
 'France',
 'Germany',
 'Ghana',
 'Greece',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Korea',
 'Latvia',
 'Luxembourg',
 'Malaysia',
 'Mauritius',
 'Mexico',
 'Mongolia',
 'Montenegro',
 'Netherlands',
 'New Zealand',
 'Norway',
 'Oman',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'United Kingdom',
 'United States'}

In [252]:
iea_slice = iea[['country', 'policy_implementation_period_start_year']]


## European Environment Agency Climate Change Mitigation Policies and Measures

### How this dataset was originated:

This database contains a number of policies and measures (PaM) implemented, adopted or planned by European countries to reduce greenhouse gas (GHG) emissions. These PaMs have been reported by Member States under the European Union (EU) Monitoring Mechanism Regulation (MMR) as recently 2017 and 2018. Member States report main characteristics of the PaMs, such as their description, objective, type, status, sectors affected, related Union Policy, entities responsible for their implementation, implementation period, etc under the reporting obligations for Greenhouse gas Monitoring Mechanism Regulation (MMR). To facilitate reporting by Member States, the EEA uses an online reporting questionnaire available via ReportNet and ‘Guidelines for reporting on policies and measures by Member States under Regulation (EU) No 525/2013 (EU Monitoring Mechanism Regulation)’<sup>1</sup>

___
<sup>1</sup>"EEA database on climate change mitigation policies and measures in Europe", European Environment Agency. Retrieved September 10, 2019 (http://pam.apps.eea.europa.eu/?source=%7B%22query%22%3A%7B%22match_all%22%3A%7B%7D%7D%2C%22display_type%22%3A%22tabular%22%2C%22sort%22%3A%5B%7B%22Country%22%3A%7B%22order%22%3A%22asc%22%7D%7D%2C%7B%22ID_of_policy_or_measure%22%3A%7B%22order%22%3A%22asc%22%7D%7D%5D%2C%22highlight%22%3A%7B%22fields%22%3A%7B%22*%22%3A%7B%7D%7D%7D%7D)


### How we got this data:

To retrieve this data we downloaded a csv file of the entire database and used pandas to convert it to a pandas dataframe, selected relevant features for our dataset and exported them to a new csv for seeding our database. 


In [109]:
#load in data from csv
eea_db = pd.read_csv('data/source/eea.csv', encoding = "utf-8")
print(eea_db.shape)
eea_db.head()

(1595, 71)


,Country,ID of policy or measure,Report ID ES,Name of policy or measure,"Single policy or measure, or group of measures",Report_ID,Policies or measures included in the group,Type of policy instrument,Status of implementation,"Policy impacting EU ETS, ESD or LULUCF emissions",...,Price reference year (realised costs),Realised benefits (EUR per tonne CO2eq reduced/ sequestered),Realised absolute benefit per year (EUR),Realised net costs (EUR per tonne CO2eq reduced/ sequestered),Realised net cost per year (EUR),Description of realised cost estimates,Reference for realised costs and benefits,Web link for realised costs and benefits,Projected net cost per year (EUR).1,Realised net cost per year (EUR).1
0,Austria,1,http://cdr.eionet.europa.eu/Converters/run_con...,EU Emission Trading Scheme (ETS),Single,2909,Single PaM,"Economic, Regulatory",Implemented,EU ETS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Austria,2,http://cdr.eionet.europa.eu/Converters/run_con...,Domestic Environmental Support Scheme,Single,2909,Single PaM,Economic,Implemented,"EU ETS, ESD",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Austria,3,http://cdr.eionet.europa.eu/Converters/run_con...,Austrian Climate and Energy Fund (KLI.EN),Single,2909,Single PaM,"Economic, Research",Implemented,"EU ETS, ESD",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Austria,4,http://cdr.eionet.europa.eu/Converters/run_con...,Increase the share of renewable energy in ener...,Single,2909,Single PaM,"Economic, Regulatory",Implemented,"EU ETS, ESD",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Austria,5,http://cdr.eionet.europa.eu/Converters/run_con...,Increase energy efficiency in energy and manuf...,Single,2909,Single PaM,"Economic, Planning, Regulatory",Implemented,"EU ETS, ESD",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
list(eea_db.columns)

['Country',
 'ID of policy or measure',
 'Report ID ES',
 'Name of policy or measure',
 'Single policy or measure, or group of measures',
 'Report_ID',
 'Policies or measures included in the group',
 'Type of policy instrument',
 'Status of implementation',
 'Policy impacting EU ETS, ESD or LULUCF emissions',
 'Sector(s) affected',
 'Objective(s)',
 'Total GHG emissions reductions in 2020 (kt CO2eq/y)',
 'Total GHG emissions reductions in 2030 (kt CO2eq/y)',
 'Entities responsible for implementing the policy (type)',
 'Entities responsible for implementing the policy',
 'Implementation period start',
 'Is the policy or measure related to a Union policy?',
 'Related Union Policy',
 'GHG(s) affected',
 'Projection scenario in which the policy or measure is included',
 'Link to national report',
 'Description',
 'Quantified objective',
 'Implementation period finish',
 'Indicator used to monitor and evaluate progress over time',
 'General comment',
 'Main reference',
 'URL to main referen

In [111]:
#drop columns
eea_db = eea_db.drop(['ID of policy or measure', 
             'Report ID ES', 
             'Single policy or measure, or group of measures', 
             'Report_ID', 
             'Projection scenario in which the policy or measure is included', 
             'Quantified objective', 
             'Indicator used to monitor and evaluate progress over time', 
             'Link to national report', 
             'General comment',
             'Main reference',
             'GHG emissions reductions EU ETS in 2020 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2020 (kt CO2eq/y)',
             'GHG emissions reductions EU ETS in 2025 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2025 (kt CO2eq/y)',
             'GHG emissions reductions EU ETS in 2030 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2030 (kt CO2eq/y)',
             'GHG emissions reductions EU ETS in 2035 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2035 (kt CO2eq/y)',
             'Reference for ex-ante assessment',
             'Web link for ex-ante assessment',
             'Year for which reduction applies (ex post)',
             'Average ex post emission reduction (kt CO2eq/y)',
             'Explanation of the basis for the mitigation estimates',
             'Factors affected by the policy or measure',
             'Reference for ex-post assessment',
             'Web link for ex-post assessment',
             'Projected costs (EUR per tonne CO2eq reduced/ sequestered)',
             'Projected absolute costs per year (EUR)',
             'Year projected cost has been calculated for',
             'Price reference year (projected costs)',
             'Projected benefits (EUR per tonne CO2eq reduced/ sequestered)',
             'Projected absolute benefit per year (EUR)',
             'Projected net costs (EUR per tonne CO2eq reduced/ sequestered)',
             'Projected net cost per year (EUR)',
             'Description of projected cost estimates',
             'Reference for projected costs and benefits',
             'Web link for projected costs and benefits',
             'Realised costs  (EUR per tonne CO2eq reduced/ sequestered)',
             'Realised absolute costs per year (EUR)',
             'Year realised cost has been calculated for',
             'Price reference year (realised costs)',
             'Realised benefits (EUR per tonne CO2eq reduced/ sequestered)',
             'Realised absolute benefit per year (EUR)',
             'Realised net costs (EUR per tonne CO2eq reduced/ sequestered)',
             'Realised net cost per year (EUR)',
             'Description of realised cost estimates',
             'Reference for realised costs and benefits',
             'Web link for realised costs and benefits',
             'Projected net cost per year (EUR).1',
             'Realised net cost per year (EUR).1',
             'Total GHG emissions reductions in 2025 (kt CO2eq/y)',
             'Total GHG emissions reductions in 2035 (kt CO2eq/y)',        
             'Objective(s)'], axis = 1)

list(eea_db.columns)


['Country',
 'Name of policy or measure',
 'Policies or measures included in the group',
 'Type of policy instrument',
 'Status of implementation',
 'Policy impacting EU ETS, ESD or LULUCF emissions',
 'Sector(s) affected',
 'Total GHG emissions reductions in 2020 (kt CO2eq/y)',
 'Total GHG emissions reductions in 2030 (kt CO2eq/y)',
 'Entities responsible for implementing the policy (type)',
 'Entities responsible for implementing the policy',
 'Implementation period start',
 'Is the policy or measure related to a Union policy?',
 'Related Union Policy',
 'GHG(s) affected',
 'Description',
 'Implementation period finish',
 'URL to main reference']

In [112]:
#rename columns for our db schema
eea_db.rename(columns={'Country': 'country',
                 'Name of policy or measure': 'name',
                 'Policies or measures included in the group': 'related_documents',
                 'Type of policy instrument': 'policy_type',
                 'Status of implementation': 'policy_status',
                 'Policy impacting EU ETS, ESD or LULUCF emissions': 'impact_on_eu_ets_esd_or_lulucf_emissions',
                 'Sector(s) affected': 'policy_target',
                 'Total GHG emissions reductions in 2020 (kt CO2eq/y)': 'total_ghg_emissions_reductions_in_2020',
                 'Total GHG emissions reductions in 2030 (kt CO2eq/y)': 'total_ghg_emissions_reductions_in_2030',
                 'Entities responsible for implementing the policy (type)': 'jurisdiction',
                 'Entities responsible for implementing the policy': 'agency',
                 'Implementation period start': 'policy_implementation_period_start_year',
                 'Is the policy or measure related to a Union policy?': 'related_to_eu_policy_boolean',
                 'Related Union Policy': 'related_to_eu_policy',
                 'GHG(s) affected': 'ghgs_affected',
                 'Description': 'description',
                 'Implementation period finish': 'policy_implementation_period_end_year',
                 'URL to main reference': 'link'}, inplace=True)

list(eea_db.columns)


['country',
 'name',
 'related_documents',
 'policy_type',
 'policy_status',
 'impact_on_eu_ets_esd_or_lulucf_emissions',
 'policy_target',
 'total_ghg_emissions_reductions_in_2020',
 'total_ghg_emissions_reductions_in_2030',
 'jurisdiction',
 'agency',
 'policy_implementation_period_start_year',
 'related_to_eu_policy_boolean',
 'related_to_eu_policy',
 'ghgs_affected',
 'description',
 'policy_implementation_period_end_year',
 'link']

In [125]:
#add in missing features
eea_db['data_source'] = 'eea'
eea_db['world_region'] = np.nan
eea_db['country_region'] = np.nan
eea_db['city'] = np.nan
eea_db['policy_written_year'] = np.nan
eea_db['impact_evaluation'] = np.nan
eea_db['policy_financing_quantity'] = np.nan
eea_db['policy_financing_period_start'] = np.nan
eea_db['policy_financing_period_end'] = np.nan
eea_db['percent_private_funded'] = np.nan
eea_db['related_to_neeap_policy'] = np.nan
eea_db['related_to_neeap_policy_boolean'] = np.nan

list(eea_db.columns)


['country',
 'name',
 'related_documents',
 'policy_type',
 'policy_status',
 'impact_on_eu_ets_esd_or_lulucf_emissions',
 'policy_target',
 'total_ghg_emissions_reductions_in_2020',
 'total_ghg_emissions_reductions_in_2030',
 'jurisdiction',
 'agency',
 'policy_implementation_period_start_year',
 'related_to_eu_policy_boolean',
 'related_to_eu_policy',
 'ghgs_affected',
 'description',
 'policy_implementation_period_end_year',
 'link',
 'data_source',
 'world_region',
 'country_region',
 'city',
 'policy_written_year',
 'impact_evaluation',
 'policy_financing_quantity',
 'policy_financing_period_start',
 'policy_financing_period_end',
 'percent_private_funded',
 'related_to_neeap_policy',
 'related_to_neeap_policy_boolean']

In [148]:
#drop columns not in schema
eea_dropped = ['ID of policy or measure', 
             'Report ID ES', 
             'Single policy or measure, or group of measures', 
             'Report_ID', 
             'Projection scenario in which the policy or measure is included', 
             'Quantified objective', 
             'Indicator used to monitor and evaluate progress over time', 
             'Link to national report', 
             'General comment',
             'Main reference',
             'GHG emissions reductions EU ETS in 2020 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2020 (kt CO2eq/y)',
             'GHG emissions reductions EU ETS in 2025 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2025 (kt CO2eq/y)',
             'GHG emissions reductions EU ETS in 2030 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2030 (kt CO2eq/y)',
             'GHG emissions reductions EU ETS in 2035 (kt CO2eq/y)',
             'GHG emissions reductions ESD in 2035 (kt CO2eq/y)',
             'Reference for ex-ante assessment',
             'Web link for ex-ante assessment',
             'Year for which reduction applies (ex post)',
             'Average ex post emission reduction (kt CO2eq/y)',
             'Explanation of the basis for the mitigation estimates',
             'Factors affected by the policy or measure',
             'Reference for ex-post assessment',
             'Web link for ex-post assessment',
             'Projected costs (EUR per tonne CO2eq reduced/ sequestered)',
             'Projected absolute costs per year (EUR)',
             'Year projected cost has been calculated for',
             'Price reference year (projected costs)',
             'Projected benefits (EUR per tonne CO2eq reduced/ sequestered)',
             'Projected absolute benefit per year (EUR)',
             'Projected net costs (EUR per tonne CO2eq reduced/ sequestered)',
             'Projected net cost per year (EUR)',
             'Description of projected cost estimates',
             'Reference for projected costs and benefits',
             'Web link for projected costs and benefits',
             'Realised costs  (EUR per tonne CO2eq reduced/ sequestered)',
             'Realised absolute costs per year (EUR)',
             'Year realised cost has been calculated for',
             'Price reference year (realised costs)',
             'Realised benefits (EUR per tonne CO2eq reduced/ sequestered)',
             'Realised absolute benefit per year (EUR)',
             'Realised net costs (EUR per tonne CO2eq reduced/ sequestered)',
             'Realised net cost per year (EUR)',
             'Description of realised cost estimates',
             'Reference for realised costs and benefits',
             'Web link for realised costs and benefits',
             'Projected net cost per year (EUR).1',
             'Realised net cost per year (EUR).1',
             'Total GHG emissions reductions in 2025 (kt CO2eq/y)',
             'Total GHG emissions reductions in 2035 (kt CO2eq/y)',        
             'Objective(s)']

print("Dropped:", ", ".join(eea_dropped))

Dropped: ID of policy or measure, Report ID ES, Single policy or measure, or group of measures, Report_ID, Projection scenario in which the policy or measure is included, Quantified objective, Indicator used to monitor and evaluate progress over time, Link to national report, General comment, Main reference, GHG emissions reductions EU ETS in 2020 (kt CO2eq/y), GHG emissions reductions ESD in 2020 (kt CO2eq/y), GHG emissions reductions EU ETS in 2025 (kt CO2eq/y), GHG emissions reductions ESD in 2025 (kt CO2eq/y), GHG emissions reductions EU ETS in 2030 (kt CO2eq/y), GHG emissions reductions ESD in 2030 (kt CO2eq/y), GHG emissions reductions EU ETS in 2035 (kt CO2eq/y), GHG emissions reductions ESD in 2035 (kt CO2eq/y), Reference for ex-ante assessment, Web link for ex-ante assessment, Year for which reduction applies (ex post), Average ex post emission reduction (kt CO2eq/y), Explanation of the basis for the mitigation estimates, Factors affected by the policy or measure, Reference fo

In [137]:
#export
eea_db.to_csv(path_or_buf='/data/eea_db.csv', index=False)

In [118]:
#test exported csv
eea = pd.read_csv('data/eea_db.csv')
eea.columns

Index(['country', 'name', 'related_documents', 'policy_type', 'policy_status',
       'impact_on_eu_ets_esd_or_lulucf_emissions', 'policy_target',
       'total_ghg_emissions_reductions_in_2020',
       'total_ghg_emissions_reductions_in_2030', 'jurisdiction', 'agency',
       'policy_implementation_period_start_year',
       'related_to_eu_policy_boolean', 'related_to_eu_policy', 'ghgs_affected',
       'description', 'policy_implementation_period_end_year', 'link',
       'data_source', 'world_region', 'country_region', 'city',
       'policy_written_year', 'impact_evaluation', 'policy_financing_quantity',
       'policy_financing_period_start', 'policy_financing_period_end',
       'percent_private_funded', 'related_to_neeap_policy',
       'related_to_neeap_policy_boolean'],
      dtype='object')

In [241]:
eea_countries = eea['country'].tolist()
set(eea_countries)

{'Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'United Kingdom'}

## MURE


### How this dataset was originated:

MURE (Mesures d'Utilisation Rationnelle de l'Energie) provides information on energy efficiency policies and measures that have been carried out in the Member States of the European Union. MURE is regularly updated by the network of national teams (once to twice a year). A network of 36 partners from 31 countries participate to the Odyssee-Mure project, usually national Efficiency Agencies or their representatives within the European network of energy efficiency agencies. 
<sup>1</sup>

___
<sup>1</sup>"Project Overview", Odyssee-MURE. Retrieved September 15, 2019 (https://www.odyssee-mure.eu/project.html)


### How we got this data:

To retrieve this data we created a csv file of the entire database using google sheets and copy and paste and then used pandas to convert it to a pandas dataframe, selected relevant features for our dataset and exported them to a new csv for seeding our database.

In [55]:
#load in data export
mure_data = pd.read_excel('data/source/muredata.xlsx')
mure_data.head()



,Code,Link,Sector,Title,Status,Type,Starting Year,Description (Open PDF)
0,HOU-AU13,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Residential building subsidy,Ongoing,Financial,1982.0,YES
1,HOU-AU27,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Energy audits (advice) for households,Ongoing,"Financial, Information/Education",1990.0,YES
2,HOU-AU5,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Minimum thermal standards for buildings,Ongoing,Legislative/Normative,1991.0,YES
3,HOU-AU18,http://www.measures-odyssee-mure.eu/output2_mr...,Household,klimaaktiv building - building standard,Ongoing,Information/Education,2005.0,YES
4,HOU-AU28,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Smart Metering and Informative Billing,Ongoing,"Financial, Information/Education",2008.0,YES


In [56]:
#get links for each entry detail
mure_links = mure_data['Link'].tolist()
len(mure_links)

2494

In [57]:
mure_links[4]

'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU28'

In [58]:
#test entry detail methodology
sample = mure_links[0]

entry = {}

get_sample = session.get(sample)

soup_s = BeautifulSoup(get_sample.text, 'html.parser')
trows = soup_s.find_all("tr",)
#trows = tables.find_all('tr')

column = [t.find_all("td") for t in trows]
for t in column:
                if len(t) == 2:
                    header = t[0].get_text()    
                    
                    for e in t[1]: 
                        text = e.get_text()
                        entry[header] = text
                        #print(header, text)

#evals = trows[:-2]
#print(evals)
#get index and merge
entry

{'Actors': 'central government, energy agencies, local government',
 'Country': 'Austria',
 'Financing': '\x80 590.000.000 from 2009 to 2015',
 'Measure Code': 'HOU-AU13',
 'Reference': 'https://www.bmlfuw.gv.at/\r\nhttp://www.bmwfw.gv.at/Seiten/default.aspx',
 'Target Audience': 'housing associations, landlords, owner-occupiers',
 'Targeted End Use': 'Total final consumption , Space heating (+elec.), Hot water (+elec.)',
 'Title': 'Residential building subsidy ',
 'Types': '21) Financial - Grants / Subsidies - For investments in new buildings exceeding building regulation; 23) Financial - Grants / Subsidies - For the purchase of more efficient boilers; 25) Financial - Grants / Subsidies - For other energy efficiency investments\xa0; 26) Financial - Grants / Subsidies - For investment in renewables'}

In [59]:
mure_paired = mure_links[0:10]
mure_paired

['http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU13',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU27',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU5',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU18',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU28',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU26',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=AU21',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=BEL19',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=BEL23',
 'http://www.measures-odyssee-mure.eu/output2_mr.asp?Cod=BEL15']

In [60]:
#use threaded approach from above to scrape each entry detail

mure_set = []

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in mure_links
            
        ]
        
        for response in await asyncio.gather(*futures):
            
            entries = dict()    
            soup_s = BeautifulSoup(response.text, 'html.parser')
            trows = soup_s.find_all("tr",)
            #trows = tables.find_all('tr')

            column = [t.find_all("td") for t in trows]
            for t in column:
                if len(t) == 2:
                    header = t[0].get_text()
                    text = t[1].get_text()

                    entries[header] = text

            entries["data_source"] = "MURE"
            
            mure_set.append(entries)
            
loop = asyncio.get_event_loop()
loop.run_until_complete(main())

In [61]:
mure_set[7]

{'Actors': 'central government, local government',
 'Country': 'Belgium',
 'Financing': '\x80 0 from  to ',
 'Measure Code': 'HOU-BEL19',
 'Reference': 'Wallonia: decree 1/05/85; Flanders: decree 1/09/92; Bruxelles : decree 1/1/2000',
 'Target Audience': 'building professions',
 'Targeted End Use': 'Space heating (+elec.)',
 'Title': 'K-level thermal regulations of residential buildings (in use prior to the EPB directive)',
 'Types': '2) Legislative/Normative - Mandatory Standards for Buildings - Minimum thermal insulation standards\xa0',
 'data_source': 'MURE'}

In [62]:
#pt in dataframe
mure_db = pd.DataFrame(mure_set)

mure_db = mure_db.rename(columns = {'Measure Code': 'Code'})
mure_db = mure_db.drop(['Title'], axis=1)

mure_db.head()

,Actors,Area,Country,Financing,Industry sector,Code,Reference,Stages,Target Audience,Targeted End Use,Tertiary sector,Types,data_source
0,"central government, energy agencies, local gov...",NaN,Austria, 590.000.000 from 2009 to 2015,NaN,HOU-AU13,https://www.bmlfuw.gv.at/\r\nhttp://www.bmwfw....,NaN,"housing associations, landlords, owner-occupiers","Total final consumption , Space heating (+elec...",NaN,21) Financial - Grants / Subsidies - For inves...,MURE
1,local government,NaN,Austria, 0 from to,NaN,HOU-AU27,"NEEAP 2014, NEEAP 2017",NaN,"building professions, housing associations, la...","Space heating (+elec.), Space cooling, Hot wat...",NaN,28) Financial - Grants / Subsidies - For energ...,MURE
2,local government,NaN,Austria, 0 from to,NaN,HOU-AU5,"Vereinbarung gem. Art. 15a B-VG, BGBl.Nr. 388/...",NaN,"building professions, housing associations, la...","Space cooling, Total final consumption , Space...",NaN,2) Legislative/Normative - Mandatory Standards...,MURE
3,"associations, central government, energy agencies",NaN,Austria, 0 from to,NaN,HOU-AU18,http://www.klimaaktiv.at/bauen-sanieren,NaN,"building professions, general public, housing ...","Space heating (+elec.), Hot water (+elec.), Sp...",NaN,36) Information/Education - Voluntary labellin...,MURE
4,utilities,NaN,Austria, 0 from to,NaN,HOU-AU28,"2nd NEEAP, NEEAP 2014, NEEAP 2017",NaN,"housing associations, landlords, owner-occupie...",Total electric consumption,NaN,30) Financial - Loans/Others - Leasing of ener...,MURE


In [63]:
mure_columns = mure_db.columns.tolist()

mure_columns

['Actors',
 'Area',
 'Country',
 'Financing',
 'Industry sector',
 'Code',
 'Reference',
 'Stages',
 'Target Audience',
 'Targeted End Use',
 'Tertiary sector',
 'Types',
 'data_source']

In [64]:
#add in detail data
mure_full = pd.merge(mure_data, mure_db[mure_columns], on='Code')

mure_full.head()

,Code,Link,Sector,Title,Status,Type,Starting Year,Description (Open PDF),Actors,Area,Country,Financing,Industry sector,Reference,Stages,Target Audience,Targeted End Use,Tertiary sector,Types,data_source
0,HOU-AU13,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Residential building subsidy,Ongoing,Financial,1982.0,YES,"central government, energy agencies, local gov...",NaN,Austria, 590.000.000 from 2009 to 2015,NaN,https://www.bmlfuw.gv.at/\r\nhttp://www.bmwfw....,NaN,"housing associations, landlords, owner-occupiers","Total final consumption , Space heating (+elec...",NaN,21) Financial - Grants / Subsidies - For inves...,MURE
1,HOU-AU27,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Energy audits (advice) for households,Ongoing,"Financial, Information/Education",1990.0,YES,local government,NaN,Austria, 0 from to,NaN,"NEEAP 2014, NEEAP 2017",NaN,"building professions, housing associations, la...","Space heating (+elec.), Space cooling, Hot wat...",NaN,28) Financial - Grants / Subsidies - For energ...,MURE
2,HOU-AU5,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Minimum thermal standards for buildings,Ongoing,Legislative/Normative,1991.0,YES,local government,NaN,Austria, 0 from to,NaN,"Vereinbarung gem. Art. 15a B-VG, BGBl.Nr. 388/...",NaN,"building professions, housing associations, la...","Space cooling, Total final consumption , Space...",NaN,2) Legislative/Normative - Mandatory Standards...,MURE
3,HOU-AU18,http://www.measures-odyssee-mure.eu/output2_mr...,Household,klimaaktiv building - building standard,Ongoing,Information/Education,2005.0,YES,"associations, central government, energy agencies",NaN,Austria, 0 from to,NaN,http://www.klimaaktiv.at/bauen-sanieren,NaN,"building professions, general public, housing ...","Space heating (+elec.), Hot water (+elec.), Sp...",NaN,36) Information/Education - Voluntary labellin...,MURE
4,HOU-AU28,http://www.measures-odyssee-mure.eu/output2_mr...,Household,Smart Metering and Informative Billing,Ongoing,"Financial, Information/Education",2008.0,YES,utilities,NaN,Austria, 0 from to,NaN,"2nd NEEAP, NEEAP 2014, NEEAP 2017",NaN,"housing associations, landlords, owner-occupie...",Total electric consumption,NaN,30) Financial - Loans/Others - Leasing of ener...,MURE


In [65]:
mure_final = mure_full.drop_duplicates(subset=None, keep='first', inplace=False)
mure_final.columns.tolist()

['Code',
 'Link',
 'Sector',
 'Title',
 'Status',
 'Type',
 'Starting Year',
 'Description (Open PDF)',
 'Actors',
 'Area',
 'Country',
 'Financing',
 'Industry sector',
 'Reference',
 'Stages',
 'Target Audience',
 'Targeted End Use',
 'Tertiary sector',
 'Types',
 'data_source']

In [66]:
##our schema columns

['country',
 'name',
 'related_documents',
 'policy_type',
 'policy_status',
 'impact_on_eu_ets_esd_or_lulucf_emissions',
 'policy_target',
 'total_ghg_emissions_reductions_in_2020',
 'total_ghg_emissions_reductions_in_2030',
 'jurisdiction',
 'agency',
 'policy_implementation_period_start_year',
 'related_to_eu_policy_boolean',
 'related_to_eu_policy',
 'ghgs_affected',
 'description',
 'policy_implementation_period_end_year',
 'link',
 'data_source',
 'world_region',
 'country_region',
 'city',
 'policy_written_year',
 'impact_evaluation',
 'policy_financing_quantity',
 'policy_financing_period_start',
 'policy_financing_period_end',
 'percent_private_funded',
 'related_to_neeap_policy',
 'related_to_neeap_policy_boolean']

## need to split finaincing into the three columns, need to do NEEAP boolean based off text search or EU boolean


mure_dat = mure_final.copy(deep=True)

mure_dat.rename(columns={'Link':'link',
                         'Industry sector':'policy_target',
                         'Title':'name',
                         'Status':'policy_status',
                         'Type':'policy_type',
                         'Starting Year':'policy_implementation_period_start_year',
                         'Actors':'jurisdiction',
                         'Country':'country',
                         'Financing':'policy_financing_quantity',
                         'Reference':'related_documents'}, inplace=True)

mure_dat.columns.tolist()

['Code',
 'link',
 'Sector',
 'name',
 'policy_status',
 'policy_type',
 'policy_implementation_period_start_year',
 'Description (Open PDF)',
 'jurisdiction',
 'Area',
 'country',
 'policy_financing_quantity',
 'policy_target',
 'related_documents',
 'Stages',
 'Target Audience',
 'Targeted End Use',
 'Tertiary sector',
 'Types',
 'data_source']

In [67]:
## drop features not in schema for now
mure_dat.drop(['Code',
                 'Sector',
                 'Description (Open PDF)',
                 'Tertiary sector',
                 'Area',
                 'Stages',
                 'Types',
                 'Targeted End Use',
                 'Target Audience'], axis=1, inplace=True)

m_drop = ['Code', 'Sector','Description (Open PDF)', 'Tertiary sector','Area','Stages','Types','Targeted End Use','Target Audience']

print("Dropped columns:\n", m_drop)

Dropped columns:
 ['Code', 'Sector', 'Description (Open PDF)', 'Tertiary sector', 'Area', 'Stages', 'Types', 'Targeted End Use', 'Target Audience']


In [68]:
#input missing features

mure_dat['agency']= np.nan
mure_dat['related_to_eu_policy_boolean'] = np.nan
mure_dat['related_to_eu_policy'] = np.nan
mure_dat['ghgs_affected'] = np.nan
mure_dat['description'] = np.nan
mure_dat['policy_implementation_period_end_year'] = np.nan
mure_dat['impact_on_eu_ets_esd_or_lulucf_emissions'] = np.nan
mure_dat['total_ghg_emissions_reductions_in_2020'] = np.nan
mure_dat['total_ghg_emissions_reductions_in_2030'] = np.nan
mure_dat['world_region'] = np.nan
mure_dat['country_region'] = np.nan
mure_dat['city'] = np.nan
mure_dat['policy_written_year'] = np.nan
mure_dat['impact_evaluation'] = np.nan
mure_dat['policy_financing_period_start'] = np.nan
mure_dat['policy_financing_period_end'] = np.nan
mure_dat['percent_private_funded'] = np.nan
mure_dat['related_to_neeap_policy'] = np.nan
mure_dat['related_to_neeap_policy_boolean'] = np.nan

mure_dat.head()

,link,name,policy_status,policy_type,policy_implementation_period_start_year,jurisdiction,country,policy_financing_quantity,policy_target,related_documents,...,world_region,country_region,city,policy_written_year,impact_evaluation,policy_financing_period_start,policy_financing_period_end,percent_private_funded,related_to_neeap_policy,related_to_neeap_policy_boolean
0,http://www.measures-odyssee-mure.eu/output2_mr...,Residential building subsidy,Ongoing,Financial,1982.0,"central government, energy agencies, local gov...",Austria, 590.000.000 from 2009 to 2015,NaN,https://www.bmlfuw.gv.at/\r\nhttp://www.bmwfw....,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://www.measures-odyssee-mure.eu/output2_mr...,Energy audits (advice) for households,Ongoing,"Financial, Information/Education",1990.0,local government,Austria, 0 from to,NaN,"NEEAP 2014, NEEAP 2017",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://www.measures-odyssee-mure.eu/output2_mr...,Minimum thermal standards for buildings,Ongoing,Legislative/Normative,1991.0,local government,Austria, 0 from to,NaN,"Vereinbarung gem. Art. 15a B-VG, BGBl.Nr. 388/...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://www.measures-odyssee-mure.eu/output2_mr...,klimaaktiv building - building standard,Ongoing,Information/Education,2005.0,"associations, central government, energy agencies",Austria, 0 from to,NaN,http://www.klimaaktiv.at/bauen-sanieren,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://www.measures-odyssee-mure.eu/output2_mr...,Smart Metering and Informative Billing,Ongoing,"Financial, Information/Education",2008.0,utilities,Austria, 0 from to,NaN,"2nd NEEAP, NEEAP 2014, NEEAP 2017",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [305]:
mure_dat.to_csv(path_or_buf='data/mure_db.csv', index=False)

In [239]:
mure_countries = mure_final['Country'].tolist()
set(mure_countries)

{'Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Estonia',
 'European Union',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Serbia',
 'Slovakia',
 'Slovenia',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom'}

In [ ]:
mure_slice = mure_final.rename

## Private Participation Renewable Energy 

### How this dataset was originated:

Private Participation Renewable Energy collects data on private participation in renewable energy in developing countries. It is part of the PPI Database and applies the same research methdology. It database relies on public sources which in most cases provide a good picture of contracts. However, in some cases they may not be accurate or contain all the required information. In such cases, the database reports the project data which seems to be the most accurate. When unable to obtain more information, projects are included with the limited information available.<sup>1</sup>

___
<sup>1</sup>"Private Participation In Renewable Energy", The World Bank. Retrieved September 27, 2019 (https://www.odyssee-mure.eu/project.html)

### How we got this data:

To retrieve this data we downloaded a csv file of the entire database and used pandas to convert it to a pandas dataframe, selected relevant features for our dataset and exported them to a new csv for seeding our database.

In [307]:
#read in data
private = pd.read_csv('data/source/privateprojects.csv', encoding = "utf-8")
print(private.shape)
private.head()

(1221, 60)


,projectID,Region,Country,IncomeGroup,IDA Status,Financial closure year,Financial closure Month,Project name,RelatedNames,Type of PPI,...,CommissioningDate,ProjectGrid,CarbonCredits,FundingYear,PrivateFunding,PublicFunding,GovtFunding,BankLocalFunding,DonorFunding,DebtEquityGrantRatio
0,4571,East Asia and Pacific,Cambodia,Low income,IDA,2001,April,Kirirom 1 and 3 Hydropower Stations,Kirium Hydro Station Project,Concession,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4571,East Asia and Pacific,Cambodia,Low income,IDA,2001,April,Kirirom 1 and 3 Hydropower Stations,Kirium Hydro Station Project,Concession,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7189,East Asia and Pacific,Cambodia,Low income,IDA,2007,April,ABC Phnom Penh Biomass Plant,NaN,Greenfield project,...,2007.0,Cambodia,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4569,East Asia and Pacific,Cambodia,Low income,IDA,2007,September,Kamchay Hydropower Station,NaN,Greenfield project,...,2009.0,Cambodia,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4573,East Asia and Pacific,Cambodia,Low income,IDA,2007,April,Stung Atay River Hydropower Station,NaN,Greenfield project,...,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [318]:
#test pulling random row
private.iloc[200]

projectID                                              3399
Region                                East Asia and Pacific
Country                                         Philippines
IncomeGroup                             Lower middle income
IDA Status                                          Non-IDA
Financial closure year                                 2004
Financial closure Month                                July
Project name                      Barit Hydroelectric Plant
RelatedNames                                            NaN
Type of PPI                                     Divestiture
Subtype of PPI                                         Full
Project status                                  Operational
Primary sector                                       Energy
Secondary sector                                        NaN
Subsector                                       Electricity
Segment                              Electricity generation
Location                             Cam

In [319]:
##rename features we found to match our schema

private.rename(columns= {'Country': 'country',
 'Financial closure year': 'policy_financing_period_start',
 'Project name' : 'name',
 'Project status': 'policy_status',
 'Primary sector': 'policy_target',
 'TerminationYear': 'policy_implementation_period_end_year',
 'GovtGrantingContract': 'jurisdiction',
 'InvestmentYear': 'policy_financing_period_start',
 'TotalInvestment': 'policy_financing_quantity',
 'CapacityYear': 'policy_implementation_period_start_year'},inplace=True)



In [325]:
### we dont actually want to drop funding year, need to resolve some of these. 
private.drop(['projectID',
       'Region',
       'IncomeGroup',
       'IDA Status',
       'Financial closure Month',
       'RelatedNames',
       'Type of PPI',
       'Subtype of PPI',
       'Secondary sector',
       'Subsector',
       'Segment',
       'Location',
       'ContractPeriod',
       'PubliclyTraded',
       'StockExchange',
       'MultipleSystems',
       'NumberOfSystems',
       'CaptiveFacility',
       'SharePercent',
       'TypeOfGovtSupport',
       'PercentPrivate',
       'GovtPaymentCommitments',
       'PhysicalAssets',
       'CapacityType',
       'Capacity',
       'Technology',
       'ContractHistory',
       'RelatedProjects',
       'BidCriteria',
       'AwardMethod',
       'NumberOfBids',
       'NumberRenewalBids',
       'Sponsors',
       'MultiLateralSupport',
       'Revenue Source',
       'RenewalBidCriteria',
       'RenewalAwardMethod',
       'DevelopmentStage',
       'CommissioningDate',
       'ProjectGrid',
       'CarbonCredits',
       'FundingYear',
       'PrivateFunding',
       'PublicFunding',
       'GovtFunding',
       'BankLocalFunding',
       'DonorFunding',
       'DebtEquityGrantRatio',
       'GovCashAssist',
       'dateStatusUpdated'
      ], axis=1, inplace=True)

In [327]:
#add missign features

private['related_documents'] = np.nan
private['policy_type'] = np.nan
private['impact_on_eu_ets_esd_or_lulucf_emissions'] = np.nan
private['total_ghg_emissions_reductions_in_2020'] = np.nan
private['total_ghg_emissions_reductions_in_2030'] = np.nan
private['agency'] = np.nan
private['related_to_eu_policy_boolean'] = np.nan
private['related_to_eu_policy'] = np.nan
private['ghgs_affected'] = np.nan
private['description'] = np.nan
private['link'] = np.nan
private['data_source'] = np.nan
private['world_region'] = np.nan
private['country_region'] = np.nan
private['city'] = np.nan
private['policy_written_year'] = np.nan
private['impact_evaluation'] = np.nan
private['policy_financing_period_end'] = np.nan
private['percent_private_funded'] = np.nan
private['related_to_neeap_policy'] = np.nan
private['related_to_neeap_policy_boolean'] = np.nan


private.columns.tolist()

['country',
 'policy_financing_period_start',
 'name',
 'policy_status',
 'policy_target',
 'policy_implementation_period_end_year',
 'jurisdiction',
 'policy_financing_period_start',
 'policy_financing_quantity',
 'policy_implementation_period_start_year',
 'related_documents',
 'policy_type',
 'impact_on_eu_ets_esd_or_lulucf_emissions',
 'total_ghg_emissions_reductions_in_2020',
 'total_ghg_emissions_reductions_in_2030',
 'agency',
 'related_to_eu_policy_boolean',
 'related_to_eu_policy',
 'ghgs_affected',
 'description',
 'link',
 'data_source',
 'world_region',
 'country_region',
 'city',
 'policy_written_year',
 'impact_evaluation',
 'policy_financing_period_end',
 'percent_private_funded',
 'related_to_neeap_policy',
 'related_to_neeap_policy_boolean']

In [328]:
private.to_csv(path_or_buf='data/private_db.csv', index=False)

## Climate Change Laws of the World


### How this dataset was originated:
Climate change Laws of the World and Climate Change Litigation of the World build on several years of data collection by both the Grantham Research Institute and the Sabin Center, including the collaboration of Grantham Institute with GLOBE International on a series of Climate Legislation Studies.<sup>1</sup>
'

As of March 2017, the climate legislation database consists of 1,261 climate change laws or policies
of similar relevance in 164 jurisdictions (163 countries and the European Union as a bloc). The
climate change legislation data are maintained and updated regularly by a team at the Grantham
Research Institute.<sup>2</sup>


### How we got this data:

We used a csv export from the database website here: http://www.lse.ac.uk/GranthamInstitute/legislation/ and then used pandas to extract relevant features to our database. 

___
<sup>1</sup>Climate Change Laws of the World database, Grantham Research Institute on Climate Change and the Environment and Sabin Center for Climate Change Law. Available at: http://www.lse.ac.uk/GranthamInstitute/legislation/.

<sup>2</sup>Global trends in climate change legislation and litigation: 2017 update. Available at: http://www.lse.ac.uk/GranthamInstitute/wp-content/uploads/2017/04/Global-trends-in-climate-change-legislation-and-litigation-WEB.pdf

In [334]:
#need to decide how to view "year passed" or think of using this in a seperate comparison type format

laws_db = pd.read_csv('data/source/legislation.csv')




Country                                                        Netherlands
Name                     National Climate Agenda: Resilient, Prosperous...
Year Passed                                                           2013
Executive/Legislative                                            Executive
Framework                                        Mitigation and adaptation
Categories               Adaptation; Carbon Pricing; Energy Demand; Ene...
Document Type                                                       Policy
Name: 500, dtype: object

In [331]:
laws_db.columns.tolist()

['Country',
 'Name',
 'Year Passed',
 'Executive/Legislative',
 'Framework',
 'Categories',
 'Document Type']

In [ ]:

laws_db.rename(columns={
    'Country':'country',
    'Name':'name',
    'Year Passed':,
    'Executive/Legislative',
    'Framework',
    'Categories',
    'Document Type'})

['country',
 'name',
 'related_documents',
 'policy_type',
 'policy_status',
 'impact_on_eu_ets_esd_or_lulucf_emissions',
 'policy_target',
 'total_ghg_emissions_reductions_in_2020',
 'total_ghg_emissions_reductions_in_2030',
 'jurisdiction',
 'agency',
 'policy_implementation_period_start_year',
 'related_to_eu_policy_boolean',
 'related_to_eu_policy',
 'ghgs_affected',
 'description',
 'policy_implementation_period_end_year',
 'link',
 'data_source',
 'world_region',
 'country_region',
 'city',
 'policy_written_year',
 'impact_evaluation',
 'policy_financing_quantity',
 'policy_financing_period_start',
 'policy_financing_period_end',
 'percent_private_funded',
 'related_to_neeap_policy',
 'related_to_neeap_policy_boolean']


## DSIRE

### How this dataset was originated:

DSIRE – the Database of State Incentives for Renewables & Efficiency – is the most comprehensive source of information on incentives and policies that support renewables and energy efficiency at the federal, state, local, and utility levels.<sup>1</sup>

DSIRE is operated by the N.C. Clean Energy Technology Center at N.C. State University and is funded by the U.S. Department of Energy. It is currently maintained and kept up to date (through 2019) by researchers at the N.C. Clean Energy Technoology

___
<sup>1</sup>“DSIRE.” NC Clean Energy Technology Center. Retrieved October 8, 2019 (https://nccleantech.ncsu.edu/resources/dsire/)

### How we got this data:
Sourcing the data from the DSRE used copy and paste for the summary table of all entries in the database into google sheets, which had individual entry urls nested. The sheet can be found here https://docs.google.com/spreadsheets/d/1ouU2GKq2jG928Ds0mez2DL0Slg3GURoW7qPua8rkGIQ/edit#gid=0. After exporting the sheet to html, the nested urls or each entry were extracted using BeautifulSoup along with other relevant features from each dataset entry. Each invidiual entry url was then scraped and its relevant features were concatenated with the corresponding data in the summary row. 

Scraping the individual entrys in the database required making a GET request for each, downloading the html, and extracting the json data used to populate the web entry.

In [5]:
#get individual links
with open("data/source/dsireindex.html", "r") as f:
    
    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

all_entries = soup.find_all("tr")
len(all_entries)


2467

In [6]:
all_entries[1:3]

[<tr style="height:20px;"><th class="row-headers-background" id="0R0" style="height: 20px;"><div class="row-header-wrapper" style="line-height: 20px;">1</div></th><td class="s0" dir="ltr"><a href="https://programs.dsireusa.org/system/program/detail/1893" target="_blank">Building Requirements for State-Funded Buildings</a></td><td class="s1" dir="ltr">NH</td><td class="s2 softmerge" dir="ltr"><div class="softmerge-inner" style="width: 99px; left: -3px;">Regulatory Policy</div></td><td class="s3 softmerge" dir="ltr"><div class="softmerge-inner" style="width: 99px; left: -3px;">Energy Standards for Public Buildings</div></td><td class="s4" dir="ltr">07/30/2006</td><td class="s4" dir="ltr">10/28/2019</td></tr>,
 <tr style="height:20px;"><th class="row-headers-background" id="0R1" style="height: 20px;"><div class="row-header-wrapper" style="line-height: 20px;">2</div></th><td class="s5" dir="ltr"><a href="https://programs.dsireusa.org/system/program/detail/85" target="_blank">Local Option -

In [11]:
#build dataset
dsr_complete = []
dsre_links = []

for i in range(1, len(all_entries)):
    dsre_entries = {}
    features = all_entries[i]
    col = features.find_all('td')
    link = col[0].find('a')['href']
    
    #add to link store
    dsre_links.append(link)
    
    #build features
    dsre_entries['country'] = 'United States'
    dsre_entries['name'] = col[0].text
    dsre_entries['related_documents'] = np.nan
    dsre_entries['policy_type'] = col[2].text
    dsre_entries['impact_on_eu_ets_esd_or_lulucf_emissions'] = np.nan
    dsre_entries['total_ghg_emissions_reductions_in_2020'] = np.nan
    dsre_entries['total_ghg_emissions_reductions_in_2030'] = np.nan
    dsre_entries['url'] = link
    dsre_entries['related_to_eu_policy_boolean'] = np.nan
    dsre_entries['related_to_eu_policy'] = np.nan
    dsre_entries['ghgs_affected'] = np.nan
    dsre_entries['data_source'] = 'DSIRE'
    dsre_entries['world_region'] = 'North America'
    dsre_entries['policy_written_year'] = np.nan
    dsre_entries['impact_evaluation'] = np.nan
    dsre_entries['policy_financing_quantity'] = np.nan
    dsre_entries['policy_financing_period_start'] = np.nan
    dsre_entries['policy_financing_period_end'] = np.nan
    dsre_entries['percent_private_funded'] = np.nan
    dsre_entries['related_to_neeap_policy'] = np.nan
    dsre_entries['related_to_neeap_policy_boolean'] = False
    dsre_entries['city'] = np.nan
    dsre_entries['policy_target'] = np.nan

    dsr_complete.append(dsre_entries)

len(dsr_complete)

2466

In [12]:
dsr_complete[0:6]

[{'city': nan,
  'country': 'United States',
  'data_source': 'DSIRE',
  'ghgs_affected': nan,
  'impact_evaluation': nan,
  'impact_on_eu_ets_esd_or_lulucf_emissions': nan,
  'name': 'Building Requirements for State-Funded Buildings',
  'percent_private_funded': nan,
  'policy_financing_period_end': nan,
  'policy_financing_period_start': nan,
  'policy_financing_quantity': nan,
  'policy_target': nan,
  'policy_type': 'Regulatory Policy',
  'policy_written_year': nan,
  'related_documents': nan,
  'related_to_eu_policy': nan,
  'related_to_eu_policy_boolean': nan,
  'related_to_neeap_policy': nan,
  'related_to_neeap_policy_boolean': False,
  'total_ghg_emissions_reductions_in_2020': nan,
  'total_ghg_emissions_reductions_in_2030': nan,
  'url': 'https://programs.dsireusa.org/system/program/detail/1893',
  'world_region': 'North America'},
 {'city': nan,
  'country': 'United States',
  'data_source': 'DSIRE',
  'ghgs_affected': nan,
  'impact_evaluation': nan,
  'impact_on_eu_ets_esd

In [21]:
#test detail

link = dsr_complete[1]['url']

dsre_entries = {}
#get individual entry details
detail = session.get(link)
soup_det = BeautifulSoup(detail.text, 'html.parser')

det_data = soup_det.find('div', attrs={"data-ng-controller": "DetailsPageCtrl"})
det_get = det_data.get('data-ng-init')[5:-1]
json_det = json.loads(det_get)

#save link for merge on link
dsre_entries['url'] = link

dsre_entries['jurisdiction'] = json_det['program']['sectorObj']['name']

if json_det['authorities'][0]['effectiveText'] is not '' or  ' ':
    dsre_entries['policy_implementation_period_start_year'] = json_det['authorities'][0]['effectiveText'][-4:]
else: 
    dsre_entries['policy_implementation_period_start_year'] = json_det['authorities'][0]['enactedText'][-4:]

if json_det['authorities'][0]['expiredText'] is not '':
    dsre_entries['policy_status'] = 'Ended'
    dsre_entries['policy_implementation_period_end_year'] = json_det['authorities'][0]['expiredText']

else:   
    dsre_entries['policy_status'] = 'Active'
    dsre_entries['policy_implementation_period_end_year'] = np.nan
    
dsre_entries['last_updated'] = json_det['program']['lastUpdated']
dsre_entries['country_region'] = json_det['program']['stateObj']['name']
dsre_entries['description'] = json_det['program']['summary']
dsre_entries['link'] = json_det['program']['websiteUrl']

dsre_entries['agency'] = json_det['program']['administrator']



In [14]:
dsre_entries

{'agency': 'Virginia Department of Mines, Minerals, and Energy',
 'country_region': 'Virginia',
 'description': '<p>&#10;&#9;The State of Virginia provides the option for any county, city, or town to exempt or partially exempt solar energy equipment and recycling equipment from local property taxes. \xa0This status is targeted toward non-commercial participants; commercial entities are fully exempt from state and local taxes under <a href="https://programs.dsireusa.org/system/program/detail/5503">Commercial Property Tax Exemption for Solar</a>.</p>&#10;<p>&#10;&#9;The solar equipment and installation has to be inspected and certified by the local building department or the Department of Environmental Quality to provide the value of the system for the purpose of determining tax credit.\xa0<span>The statute broadly defines solar energy equipment as any that is &#34;designed and used primarily for the purpose of collecting, generating, transferring, or storing thermal or electric energy.&

In [29]:
#use threaded approach like above to compile all entries
det = []

async def main():
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:

        loop = asyncio.get_event_loop()
        futures = [
            loop.run_in_executor(
                executor, 
                session.get, 
                link
            )
            for link in dsre_links
        ]
        
        for response in await asyncio.gather(*futures):
            
            dsre_entries = {}
            #get individual entry details
            soup_det = BeautifulSoup(response.text, 'html.parser')

            det_data = soup_det.find('div', attrs={"data-ng-controller": "DetailsPageCtrl"})
            det_get = det_data.get('data-ng-init')[5:-1]
            json_det = json.loads(det_get)

            #save link for merge on link
            dsre_entries['url'] = link

            dsre_entries['jurisdiction'] = json_det['program']['sectorObj']['name']
            
            if len(json_det['authorities']) == 0:
                dsre_entries['policy_implementation_period_start_year'] = np.nan
                dsre_entries['policy_status'] = np.nan
                dsre_entries['policy_implementation_period_end_year'] = np.nan
                
            
            else:

                if json_det['authorities'][0]['effectiveText'] is not '' or ' ':
                    dsre_entries['policy_implementation_period_start_year'] = json_det['authorities'][0]['effectiveText']
                else: 
                    dsre_entries['policy_implementation_period_start_year'] = json_det['authorities'][0]['enactedText']

                if json_det['authorities'][0]['expiredText'] is not '':
                    dsre_entries['policy_status'] = 'Ended'
                    dsre_entries['policy_implementation_period_end_year'] = json_det['authorities'][0]['expiredText']

                else:   
                    dsre_entries['policy_status'] = 'Active'
                    dsre_entries['policy_implementation_period_end_year'] = np.nan

            dsre_entries['last_updated'] = json_det['program']['lastUpdated']
            dsre_entries['country_region'] = json_det['program']['stateObj']['name']
            dsre_entries['description'] = json_det['program']['summary']
            dsre_entries['link'] = json_det['program']['websiteUrl']

            dsre_entries['agency'] = json_det['program']['administrator']
            
            det.append(dsre_entries)


loop = asyncio.get_event_loop()            
loop.run_until_complete(main())

In [30]:
#should match the number in the summary list
print(len(dsr_complete))
print(len(det))

2466
2466


In [31]:
#merge details with summary entries (on url) and drop last update and link we merged on
sum_db = pd.DataFrame(dsr_complete)
det_db = pd.DataFrame(det)

In [35]:
sum_db.head()

,city,country,data_source,ghgs_affected,impact_evaluation,impact_on_eu_ets_esd_or_lulucf_emissions,name,percent_private_funded,policy_financing_period_end,policy_financing_period_start,...,policy_written_year,related_documents,related_to_eu_policy,related_to_eu_policy_boolean,related_to_neeap_policy,related_to_neeap_policy_boolean,total_ghg_emissions_reductions_in_2020,total_ghg_emissions_reductions_in_2030,url,world_region
0,NaN,United States,DSIRE,NaN,NaN,NaN,Building Requirements for State-Funded Buildings,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,https://programs.dsireusa.org/system/program/d...,North America
1,NaN,United States,DSIRE,NaN,NaN,NaN,Local Option - Residential Property Tax Exempt...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,https://programs.dsireusa.org/system/program/d...,North America
2,NaN,United States,DSIRE,NaN,NaN,NaN,Town of Chapel Hill - Energy Conservation Requ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,https://programs.dsireusa.org/system/program/d...,North America
3,NaN,United States,DSIRE,NaN,NaN,NaN,High-Performance Building Requirements for Sta...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,https://programs.dsireusa.org/system/program/d...,North America
4,NaN,United States,DSIRE,NaN,NaN,NaN,City of St. Louis - Green Building Standard,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,https://programs.dsireusa.org/system/program/d...,North America


In [36]:
det_db.head()

,agency,country_region,description,jurisdiction,last_updated,link,policy_implementation_period_end_year,policy_implementation_period_start_year,policy_status,url
0,None,New Hampshire,<p>New Hampshire enacted legislation (S.B. 409...,State,"October 28, 2019",https://das.nh.gov/publicworks/High%20Performa...,NaN,07/01/2011,Active,https://programs.dsireusa.org/system/program/d...
1,"Virginia Department of Mines, Minerals, and En...",Virginia,<p>&#10;&#9;The State of Virginia provides the...,State,"October 11, 2019",http://www.dmme.virginia.gov/DE/Energy_Incenti...,NaN,,Active,https://programs.dsireusa.org/system/program/d...
2,None,North Carolina,<p>The Town of Chapel Hill’s energy-conservati...,Local,"October 7, 2019",,NaN,,Active,https://programs.dsireusa.org/system/program/d...
3,None,South Dakota,"<p>&#10;&#9;In March 2008, South Dakota enacte...",State,"September 30, 2019",,NaN,,Active,https://programs.dsireusa.org/system/program/d...
4,Board of Public Service,Missouri,<p>This ordinance requires LEED (Leadership in...,Local,"September 30, 2019",https://www.stlouis-mo.gov/sustainability/,None,None,Ended,https://programs.dsireusa.org/system/program/d...


In [32]:
full_dsre = sum_db.merge(det_db, on='url',)

In [69]:
#set(full_dsre['name'].tolist())

In [ ]:
dsre_full.to_csv(path_or_buf='data/dsire_db.csv', index=False)